In [1]:
# !git clone https://github.com/kamalkraj/minGPT-TF.git

fatal: destination path 'minGPT-TF' already exists and is not an empty directory.


In [1]:
! pip install fastprogress==0.2.3

In [2]:
import os
os.chdir('minGPT-TF')

In [3]:
import math
import numpy as np
import tensorflow as tf
from mingpt.model import GPT, GPTConfig

In [4]:
class CharDataset:

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return math.ceil(len(self.data) / (self.block_size + 1))

    def __iter__(self):
        # we're actually going to "cheat" and pick a spot in the dataset at random
        for _ in range(self.__len__()):
            i = np.random.randint(0, len(self.data) - (self.block_size + 1))
            chunk = self.data[i:i+self.block_size+1]
            dix = [self.stoi[s] for s in chunk]
            x = tf.convert_to_tensor(dix[:-1], dtype=tf.int32)
            y = tf.convert_to_tensor(dix[1:], dtype=tf.int32)
            yield x, y
    
    __call__ = __iter__

In [5]:
block_size = 128 

In [6]:
filename = 'Voyage.txt'

text = open(filename, 'r').read()
train_dataset_gen = CharDataset(text, block_size) 

data has 716355 characters, 86 unique.


In [7]:
train_dataset = tf.data.Dataset.from_generator(train_dataset_gen,(tf.int32,tf.int32))

In [8]:
from mingpt.model import GPT, GPTConfig
mconf = GPTConfig(train_dataset_gen.vocab_size, train_dataset_gen.block_size,
                  n_layer=8, n_head=8, n_embd=512)

In [9]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=200, batch_size=32, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=200*len(train_dataset_gen)*block_size,
                      num_workers=4)
trainer = Trainer(GPT, mconf, train_dataset, len(train_dataset_gen), None, None, tconf)

In [10]:
trainer.train()

epoch 1: train loss 338.58279. lr 5.999641e-04
epoch 2: train loss 295.10031. lr 5.998540e-04
epoch 3: train loss 257.28720. lr 5.996701e-04
epoch 4: train loss 231.68773. lr 5.994122e-04
epoch 5: train loss 213.85141. lr 5.990804e-04
epoch 6: train loss 200.20932. lr 5.986748e-04
epoch 7: train loss 190.81889. lr 5.981955e-04
epoch 8: train loss 183.01021. lr 5.976426e-04
epoch 9: train loss 177.05498. lr 5.970162e-04
epoch 10: train loss 172.11165. lr 5.963166e-04
epoch 11: train loss 167.42877. lr 5.955438e-04
epoch 12: train loss 163.84512. lr 5.946981e-04
epoch 13: train loss 160.00409. lr 5.937797e-04
epoch 14: train loss 156.35429. lr 5.927888e-04
epoch 15: train loss 154.23540. lr 5.917256e-04
epoch 16: train loss 152.00546. lr 5.905905e-04
epoch 17: train loss 149.17911. lr 5.893836e-04
epoch 18: train loss 146.25595. lr 5.881054e-04
epoch 19: train loss 144.17046. lr 5.867560e-04
epoch 20: train loss 142.38054. lr 5.853358e-04
epoch 21: train loss 139.61748. lr 5.838453e-04
e

In [13]:
from mingpt.utils import sample

context = "You think pretty smart"
x = tf.convert_to_tensor([train_dataset_gen.stoi[s] for s in context], dtype=tf.int32)[None,...]
y = sample(trainer.model, x, 2000, temperature=0.95, sample=True, top_k=5)[0]
completion = ''.join([train_dataset_gen.itos[int(i)] for i in y])
print(completion)

You think pretty smart."
    "~  . " " Jay, on the snat just on the behind "family."
"A real pro burglar Terry exclaimed. "You son-of-a-gun." Hanlon said admiringly.
   "Son-of-a-something, anyway," Anita added. They all laughed.
   Sirocco had already known the story, but she had managed to scrape together a quorum. He took the first of his surface of Chiron in the event of hostilities, and the intensive training in antiterrorist and counterguerilla operations that had been initiated. The speech reminded him of the old-time slave ships which arrived carrying messages of brotherhood and little with detachable in his voice. "Family; it might was an interesting things to do, and ahead lay the challenge by the door; Driscoll opened the latches and swung the door outward while Colman held his breath. The alarms remained quiet into reptoin to gesture.
	He glanced forward ante seconds that the Chironians were paying for it now, and the fools who had pushed it in open in the shortest time for